In [1]:
import pandas as pd
import numpy as np
#import sframe
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import json
from __future__ import division

In [2]:
dtype_dict = {'id':int, 'member_Id':int, 'loan_amnt':float, 'funded_amnt':float, 'funded_amnt_inv':float, 
              'term':str, 'float_rate':float, 'installment':float, 'grade':str, 'sub_grade':str, 'emp_title':str,
              'emp_length':str, 'home_ownership':str, 'annual_inc':float, 'is_inc_v':str, 'issue_d':str, 'loan_status':str, 
              'pymnt_plan':str, 'url':str, 'desc':str, 'purpose':str,'title':str,'zipcode':str,'addr_state':str,'dti':float,'deling_2yrs':float,'earliest_cr_line':str,
             'inq_last_6mths':float,'mths_since_last_delinq':float,'mths_since_last_record':float,'open_acc':float,'pub_rec':float,
             'revol_bal':float,'revol_util':float,'total_acc':float,'initial_list_status':str,'out_prncp':float,'out_prncp_inv':float,
             'total_pymnt':float,'total_pymnt_inv':float,'total_rec_prncp':float,'total_rec_float':float,'total_rec_late_fee':float,
              'recoveries':float,'collection_recovery_fee':float,'last_pymnt_d':str,'last_pymnt_amnt':float,'next_pymnt_d':str,
              'last_credit_pull_d':str,'collections_12_mths_ex_med':float,'mths_since_last_major_derog':float,
              'policy_code':float,'not_compliant':float,'status':str,'inactive_loans':float,'bad_loans':int,'emp_length_num':float,
              'grade_num':float,'sub_grade_num':float,'delinq_2yrs_zero':float,'pub_rec_zero':float,'collections_12_mths_zero':float,
              'short_emp':float,'payment_inc_ratio':float,'final_d':str,'last_delinq_none':float,'last_record_none':float,'last_major_derog_none':float}
#str,float,int

In [4]:
loans = pd.read_csv("/Users/wxu/workspace/ML-UW-Coursera/classification/week4/lending-club-data.csv",header=0,dtype = dtype_dict)
loans["safe_loans"] = loans['bad_loans'].apply(lambda x: 1 if x==0 else -1)
loans.drop("bad_loans",1)
loans.dtypes

In [33]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length'         # number of years of employment
           ]

target = 'safe_loans'
selected = features+[target]



['grade', 'term', 'home_ownership', 'emp_length', 'safe_loans']


In [34]:
loansSelectedFeature = loans[selected]


In [35]:
loansWithDummy = pd.get_dummies(loansSelectedFeature,columns=features)
loansWithDummy = loansWithDummy.fillna(0)

In [41]:
loansWithDummy.shape[0]

122607

In [39]:
with open("/Users/wxu/workspace/ML-UW-Coursera/classification/week3/trainingIdx.json") as myjson:
    for line in myjson:
        trainIdx = json.loads(line)
with open("/Users/wxu/workspace/ML-UW-Coursera/classification/week3/validIdx.json") as myjson:
    for line in myjson:
        validIdx = json.loads(line)
train = loansWithDummy.iloc[trainIdx]
valid = loansWithDummy.iloc[validIdx]

In [56]:
dummyFeatures = list(train.columns)
dummyFeatures.remove("safe_loans")
len(dummyFeatures)

25

In [51]:
#Early stopping condition 2: Minimum node size
def reached_minimum_node_size(data, min_node_size):
    if len(data)<= min_node_size: return True 
    else : return False
   

In [65]:
def error_reduction(error_before_split, error_after_split):
    return error_before_split-error_after_split

In [61]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # Count the number of 1's (safe loans)
    ## YOUR CODE HERE
    num_of_safe = (labels_in_node==1).sum()
    # Count the number of -1's (risky loans)
    ## YOUR CODE HERE
    num_of_risk= (labels_in_node==-1).sum()            
    # Return the number of mistakes that the majority classifier makes.
    ## YOUR CODE HERE
    if num_of_safe>=num_of_risk:return num_of_risk
    else: return num_of_safe
    

In [62]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split =  data[data[feature]==1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        left_mistakes = intermediate_node_num_mistakes(left_split[target])            

        # Calculate the number of misclassified examples in the right split.
        ## YOUR CODE HERE
        right_mistakes = intermediate_node_num_mistakes(right_split[target]) 
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        ## YOUR CODE HERE
        error = (left_mistakes+right_mistakes)/num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_feature = feature
            best_error = error
        
    
    return best_feature # Return the best feature we found

In [47]:
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True    }   ## YOUR CODE HERE 
   
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1         
    else:
        leaf['prediction'] = -1         ## YOUR CODE HERE        

    # Return the leaf node
    return leaf 

In [66]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10,min_node_size=1,min_error_reduction=0.0):
    remaining_features = features[:] #how to make a copy
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
     # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if  intermediate_node_num_mistakes(target_values)== 0:  ## YOUR CODE HERE
        print "Stopping condition 1 reached."     
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if remaining_features == None :   ## YOUR CODE HERE
        print "Stopping condition 2 reached."    
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # early stoping condition 1 (limit tree depth)
    if current_depth >= max_depth:  ## YOUR CODE HERE
        print "Reached maximum depth. Stopping for now."
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)
    # early stopping condition 2
    if reached_minimum_node_size(target_values, min_node_size):
        print  "Early stopping condition 2 reached. Reached minimum node size."
        return create_leaf(target_values)
   
    
    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    ## YOUR CODE HERE
    
    splitting_feature = best_splitting_feature(data, features, target)
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split =  data[data[splitting_feature] == 1]   
    
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes = intermediate_node_num_mistakes(left_split[target])
    right_mistakes = intermediate_node_num_mistakes(right_split[target])
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if error_reduction(error_before_split, error_after_split) < min_error_reduction:
        print "Early stopping condition 3 reached. Minimum error reduction."
        return  create_leaf(target_values)
    
    
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print "Creating leaf node."
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print "Creating leaf node."
        return create_leaf(right_split[target])
     # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [67]:
my_decision_tree_new = decision_tree_create(train, dummyFeatures, 'safe_loans', max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
R

In [70]:
my_decision_tree_old = decision_tree_create(train, dummyFeatures, 'safe_loans', max_depth = 6,min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
R

In [71]:
def classify(tree, x, annotate = False):
       # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [77]:
print valid.iloc[0]
print 'Predicted class: %s ' % classify(my_decision_tree_new, valid.iloc[0])

safe_loans                -1
grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            0
term_ 60 months            1
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
emp_length_n/a             0
Name: 24, dtype: float64
Predicted class: -1 


In [79]:
classify(my_decision_tree_new, valid.iloc[0], annotate = True)

Split on term_ 36 months = 0.0
Split on grade_A = 0.0
Split on grade_B = 0.0
Split on grade_C = 0.0
Split on grade_D = 1.0
At leaf, predicting -1


-1

In [78]:
classify(my_decision_tree_old, valid.iloc[0], annotate = True)

Split on term_ 36 months = 0.0
Split on grade_A = 0.0
Split on grade_B = 0.0
Split on grade_C = 0.0
Split on grade_D = 1.0
At leaf, predicting -1


-1

In [81]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x),axis=1)
    
    # Once you've made the predictions, calculate the classification error and return it
    acc = (prediction==data['safe_loans']).sum()/len(data)
    return 1-acc
    ## YOUR CODE HERE

In [82]:
evaluate_classification_error(my_decision_tree_new, valid)

0.38377854373115039

In [83]:
evaluate_classification_error(my_decision_tree_old, valid)

0.38377854373115039

In [84]:
model1 = decision_tree_create(train, dummyFeatures, 'safe_loans', max_depth = 2,min_node_size = 0, min_error_reduction=-1)
model2 = decision_tree_create(train, dummyFeatures, 'safe_loans', max_depth = 6,min_node_size = 0, min_error_reduction=-1)
model3 = decision_tree_create(train, dummyFeatures, 'safe_loans', max_depth = 14,min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
Split on feature grade_D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (4701 data po

In [86]:
print "Training data, classification error (model 1):", evaluate_classification_error(model1, train)
print "Training data, classification error (model 2):", evaluate_classification_error(model2, train)
print "Training data, classification error (model 3):", evaluate_classification_error(model3, train)
print "Validation data, classification error (model 1):", evaluate_classification_error(model1, valid)
print "Validation data, classification error (model 2):", evaluate_classification_error(model2, valid)
print "Validation data, classification error (model 3):", evaluate_classification_error(model3, valid)

 Training data, classification error (model 1): 0.400037610144
Training data, classification error (model 2): 0.381850419084
Training data, classification error (model 3): 0.376182033097
Validation data, classification error (model 1): 0.398104265403
Validation data, classification error (model 2): 0.383778543731
Validation data, classification error (model 3): 0.37731581215


In [87]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

In [88]:
print count_leaves(model1)
print count_leaves(model2)
print count_leaves(model3)

4
19
41


In [94]:
model4 = decision_tree_create(train, dummyFeatures, 'safe_loans', max_depth = 6,min_node_size = 0, min_error_reduction=-1)
model5 = decision_tree_create(train, dummyFeatures, 'safe_loans', max_depth = 6,min_node_size = 0, min_error_reduction=0.0)
model6 = decision_tree_create(train, dummyFeatures, 'safe_loans', max_depth = 6,min_node_size = 0, min_error_reduction=5.0)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
R

In [95]:
print "Validation data, classification error (model 1):", evaluate_classification_error(model4, valid)
print "Validation data, classification error (model 2):", evaluate_classification_error(model5, valid)
print "Validation data, classification error (model 3):", evaluate_classification_error(model6, valid)
print count_leaves(model4)
print count_leaves(model5)
print count_leaves(model6)

Validation data, classification error (model 1): 0.383778543731
Validation data, classification error (model 2): 0.383778543731
Validation data, classification error (model 3): 0.503446790177
19
19
1


In [91]:
model7 = decision_tree_create(train, dummyFeatures, 'safe_loans', max_depth = 6,min_node_size = 0, min_error_reduction=-1)
model8 = decision_tree_create(train, dummyFeatures, 'safe_loans', max_depth = 6,min_node_size = 2000, min_error_reduction=-1)
model9 = decision_tree_create(train, dummyFeatures, 'safe_loans', max_depth = 6,min_node_size = 50000, min_error_reduction=-1)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
R

In [93]:
print "Validation data, classification error (model 1):", evaluate_classification_error(model7, valid)
print "Validation data, classification error (model 2):", evaluate_classification_error(model8, valid)
print "Validation data, classification error (model 3):", evaluate_classification_error(model9, valid)
print count_leaves(model7)
print count_leaves(model8)
print count_leaves(model9)

Validation data, classification error (model 1): 0.383778543731
Validation data, classification error (model 2): 0.383778543731
Validation data, classification error (model 3): 0.503446790177
19
19
1
